In [1]:
from queue import Queue
import threading
from socketIO_client import SocketIO, BaseNamespace
import numpy as np
import time

In [2]:
def run_parallel_in_threads(target, args_list):
    globalparas = []
    result = Queue()
    def task_wrapper(*args):
        result.put(target(*args))
    threads = [threading.Thread(target = task_wrapper, args = args) for args in args_list]
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    while not result.empty():
        globalparas.append(result.get())
    globalparas = list(filter(None, globalparas))
    return globalparas

def get_time_5001(i):
    last_time = time.time()
    live_namespace = SocketIO('http://localhost', 5001).define(BaseNamespace, '/live')
    live_namespace.emit('event',{'id':int(i)})
    return [time.time()-last_time, i]

def get_time_5000(i):
    last_time = time.time()
    live_namespace = SocketIO('http://localhost', 5000).define(BaseNamespace, '/live')
    live_namespace.emit('event',{'id':int(i)})
    return [time.time()-last_time, i]

def get_time_80(i):
    last_time = time.time()
    live_namespace = SocketIO('http://localhost/socket.io', 80).define(BaseNamespace, '/live')
    live_namespace.emit('event',{'id':int(i)})
    return [time.time()-last_time, i]

In [4]:
live_namespace = SocketIO('http://localhost/socket.io', 80).define(BaseNamespace, '/live')
live_namespace.emit('event',{'id':int(0)})

In [5]:
def stress_test(socketio, count, per_second):
    aranged = np.arange(count)
    for index in range(0,aranged.shape[0],per_second):
        batch = aranged[index:min(index + per_second,aranged.shape[0])]
        threads = []
        for k in batch:
            threads.append((k,))
        outputs = run_parallel_in_threads(socketio, threads)
        total = 0
        for i in outputs:
            total += float(i[0])
        print('index %d, total time taken %f s, average time taken %f s'%(index, total, total / len(outputs)))
        time.sleep(2)

In [6]:
stress_test(get_time_80, 50,10)

index 0, total time taken 1.087309 s, average time taken 0.108731 s
index 10, total time taken 1.203958 s, average time taken 0.120396 s
index 20, total time taken 1.310126 s, average time taken 0.131013 s
index 30, total time taken 1.595863 s, average time taken 0.159586 s
index 40, total time taken 1.548332 s, average time taken 0.154833 s
